In [12]:
import pandas as pd
import trino
import json

# Cấu hình để Pandas hiển thị đầy đủ nội dung (quan trọng khi xem JSON)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

print("Các thư viện đã sẵn sàng.")

Các thư viện đã sẵn sàng.


In [13]:
# Kết nối tới Trino service
conn = trino.dbapi.connect(
    host="127.0.0.1",
    port=8083,
    user="don",
    catalog="lakehouse",   # hoặc "iceberg" tùy SHOW CATALOGS
    schema="rva",       # đây là database trong iceberg
)


cursor = conn.cursor()
print(f"Kết nối Trino (Catalog: {conn.catalog}, Schema: {conn.schema}) thành công!")

Kết nối Trino (Catalog: lakehouse, Schema: rva) thành công!


In [14]:
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()
print("Các bảng có trong schema 'retail':")
print(tables)

# Kết quả mong đợi: [('bronze_raw',)]

Các bảng có trong schema 'retail':
[['bronze_raw'], ['gold_people_per_minute'], ['gold_track_summary'], ['gold_zone_dwell'], ['gold_zone_heatmap'], ['silver_detections_v2'], ['silver_detections_v3']]


#### Đọc dữ liệu Bronze bằng Trino

In [15]:
cursor.execute("SELECT * FROM bronze_raw LIMIT 10")
rows = cursor.fetchall()
print(rows)


[['v1', None, None, '{"schema_version": "1.0", "pipeline_run_id": "1a70d903a6e147a187bd28d0cd31f54c", "source": {"store_id": "store_01", "camera_id": "cam_01", "stream_id": "stream_01"}, "frame_index": 1, "capture_ts": "2025-11-24T12:00:25.321241+00:00", "image_size": {"width": 1920, "height": 1080}, "detections": [{"det_id": "1-0", "class": "person", "class_id": 0, "conf": 0.8935845494270325, "bbox": {"x1": 1001.053955078125, "y1": 255.43377685546875, "x2": 1156.245361328125, "y2": 677.5283203125}, "bbox_norm": {"x": 0.5213822682698568, "y": 0.23651275634765626, "w": 0.080828857421875, "h": 0.3908282809787326}, "centroid": {"x": 1078, "y": 466}, "centroid_norm": {"x": 0.5617966969807943, "y": 0.43192689683702257}, "track_id": 1}, {"det_id": "1-1", "class": "person", "class_id": 0, "conf": 0.6988767981529236, "bbox": {"x1": 1216.485595703125, "y1": 294.24261474609375, "x2": 1359.431396484375, "y2": 746.9419555664062}, "bbox_norm": {"x": 0.6335862477620443, "y": 0.27244686550564234, "w"

#### Kiểm tra cấu trúc bảng (Schema)

In [16]:
cursor.execute("DESCRIBE bronze_raw")
columns = cursor.fetchall()
print("Cấu trúc bảng bronze_raw:")
pd.DataFrame(columns, columns=['Column', 'Type', 'Extra', 'Comment'])

Cấu trúc bảng bronze_raw:


,Column,Type,Extra,Comment
0,schema_version,varchar,,
1,pipeline_run_id,varchar,,
2,frame_index,bigint,,
3,payload,varchar,,
4,camera_id,varchar,,
5,store_id,varchar,,
6,ingest_ts,timestamp(6),,


#### Số lượng record trong Bronze

In [17]:

cursor.execute("SELECT COUNT(*) FROM bronze_raw")
rows = cursor.fetchall()
print(rows)


[[464]]


#### Xem schema bảng bronze_raw

In [18]:
cursor.execute("DESCRIBE bronze_raw")
schema_rows = cursor.fetchall()
for r in schema_rows:
    print(r)


['schema_version', 'varchar', '', '']
['pipeline_run_id', 'varchar', '', '']
['frame_index', 'bigint', '', '']
['payload', 'varchar', '', '']
['camera_id', 'varchar', '', '']
['store_id', 'varchar', '', '']
['ingest_ts', 'timestamp(6)', '', '']


1) schema_version (varchar)

Phiên bản schema của metadata JSON từ module vision.
Giúp bạn biết lúc nào format JSON thay đổi.
Ví dụ: "v1".

2) pipeline_run_id (varchar)

ID duy nhất cho mỗi lần chạy vision pipeline.
Một video chạy lại → một pipeline_run_id khác.
Dùng để debug hoặc trace theo từng lần chạy.

3) frame_index (bigint)

Số thứ tự frame trong video.
Ví dụ: 1, 2, 3, 4…
Quan trọng cho tracking thời gian thực.

4) payload (varchar)

JSON thô từ vision.
Đây là phần to nhất: detections, bbox, timestamp, centroid…
Silver sẽ parse từ đây.

5) camera_id (varchar)

ID camera gửi dữ liệu (ví dụ: "cam_01").
Bạn dùng để partition/cluster data theo camera.

6) store_id (varchar)

ID cửa hàng (ví dụ "store_01").
Giúp truy vấn BI theo từng cửa hàng.

7) ingest_ts (timestamp(6))

Thời điểm Flink ghi record vào Iceberg.
Không phải capture_ts từ JSON → mà là timestamp ingestion.
Dùng để kiểm tra trễ, latency pipeline.

---

In [19]:
cursor.execute("""
SELECT 
    json_extract(payload, '$.capture_ts') AS ts,
    json_extract(payload, '$.detections') AS dets
FROM bronze_raw
LIMIT 3
""")
print(cursor.fetchall())


[['"2025-11-24T12:00:25.321241+00:00"', '[{"det_id":"1-0","class":"person","class_id":0,"conf":0.8935845494270325,"bbox":{"x1":1001.053955078125,"y1":255.43377685546875,"x2":1156.245361328125,"y2":677.5283203125},"bbox_norm":{"x":0.5213822682698568,"y":0.23651275634765626,"w":0.080828857421875,"h":0.3908282809787326},"centroid":{"x":1078,"y":466},"centroid_norm":{"x":0.5617966969807943,"y":0.43192689683702257},"track_id":1},{"det_id":"1-1","class":"person","class_id":0,"conf":0.6988767981529236,"bbox":{"x1":1216.485595703125,"y1":294.24261474609375,"x2":1359.431396484375,"y2":746.9419555664062},"bbox_norm":{"x":0.6335862477620443,"y":0.27244686550564234,"w":0.07445093790690104,"h":0.4191660563151042},"centroid":{"x":1287,"y":520},"centroid_norm":{"x":0.6708117167154948,"y":0.48202989366319443},"track_id":2},{"det_id":"1-2","class":"person","class_id":0,"conf":0.6859533786773682,"bbox":{"x1":680.6466064453125,"y1":53.300048828125,"x2":745.2310180664062,"y2":210.07730102539062},"bbox_nor

### Lớp silver

#### Lấy capture_ts + số người / frame

In [20]:
cursor.execute("""
SELECT
    json_extract_scalar(payload, '$.frame_index') AS frame_index,
    json_extract_scalar(payload, '$.capture_ts') AS capture_ts_str,
    json_array_length(json_extract(payload, '$.detections')) AS num_person,
    camera_id,
    store_id,
    ingest_ts
FROM bronze_raw
LIMIT 5
""")
rows = cursor.fetchall()
for r in rows:
    print(r)


['2', '2025-11-24T12:00:25.517747+00:00', 4, 'cam_01', 'store_01', datetime.datetime(2025, 11, 24, 11, 58, 38, 911000)]
['3', '2025-11-24T12:00:25.569256+00:00', 5, 'cam_01', 'store_01', datetime.datetime(2025, 11, 24, 11, 58, 38, 914000)]
['1', '2025-11-24T12:00:25.321241+00:00', 4, 'cam_01', 'store_01', datetime.datetime(2025, 11, 24, 11, 58, 38, 164000)]
['4', '2025-11-24T12:00:25.614361+00:00', 5, 'cam_01', 'store_01', datetime.datetime(2025, 11, 24, 11, 58, 38, 917000)]
['5', '2025-11-24T12:00:25.671653+00:00', 5, 'cam_01', 'store_01', datetime.datetime(2025, 11, 24, 11, 58, 38, 919000)]


#### Bung mảng detections thành từng dòng (UNNEST)

In [21]:
cursor.execute("""
SELECT
    CAST(json_extract_scalar(payload, '$.frame_index') AS bigint) AS frame_index,
    from_iso8601_timestamp(json_extract_scalar(payload, '$.capture_ts')) AS capture_ts,
    camera_id,
    store_id,
    json_extract_scalar(det, '$.det_id') AS det_id,
    json_extract_scalar(det, '$.class') AS class,
    CAST(json_extract_scalar(det, '$.class_id') AS integer) AS class_id,
    CAST(json_extract_scalar(det, '$.conf') AS double) AS conf,
    CAST(json_extract_scalar(det, '$.bbox.x1') AS double) AS bbox_x1,
    CAST(json_extract_scalar(det, '$.bbox.y1') AS double) AS bbox_y1,
    CAST(json_extract_scalar(det, '$.bbox.x2') AS double) AS bbox_x2,
    CAST(json_extract_scalar(det, '$.bbox.y2') AS double) AS bbox_y2,
    CAST(json_extract_scalar(det, '$.centroid.x') AS double) AS centroid_x,
    CAST(json_extract_scalar(det, '$.centroid.y') AS double) AS centroid_y,
    CAST(json_extract_scalar(det, '$.track_id') AS bigint) AS track_id,
    ingest_ts
FROM bronze_raw
CROSS JOIN UNNEST(
    CAST(json_extract(payload, '$.detections') AS array(json))
) AS t(det)
LIMIT 10
""")
rows = cursor.fetchall()
for r in rows:
    print(r)


[1, datetime.datetime(2025, 11, 24, 12, 0, 25, 321000, tzinfo=zoneinfo.ZoneInfo(key='UTC')), 'cam_01', 'store_01', '1-0', 'person', 0, 0.8935845494270325, 1001.053955078125, 255.43377685546875, 1156.245361328125, 677.5283203125, 1078.0, 466.0, 1, datetime.datetime(2025, 11, 24, 11, 58, 38, 164000)]
[1, datetime.datetime(2025, 11, 24, 12, 0, 25, 321000, tzinfo=zoneinfo.ZoneInfo(key='UTC')), 'cam_01', 'store_01', '1-1', 'person', 0, 0.6988767981529236, 1216.485595703125, 294.24261474609375, 1359.431396484375, 746.9419555664062, 1287.0, 520.0, 2, datetime.datetime(2025, 11, 24, 11, 58, 38, 164000)]
[1, datetime.datetime(2025, 11, 24, 12, 0, 25, 321000, tzinfo=zoneinfo.ZoneInfo(key='UTC')), 'cam_01', 'store_01', '1-2', 'person', 0, 0.6859533786773682, 680.6466064453125, 53.300048828125, 745.2310180664062, 210.07730102539062, 712.0, 131.0, 3, datetime.datetime(2025, 11, 24, 11, 58, 38, 164000)]
[1, datetime.datetime(2025, 11, 24, 12, 0, 25, 321000, tzinfo=zoneinfo.ZoneInfo(key='UTC')), 'cam

#### Tạo bảng Silver trong Iceberg (Trino)

In [22]:
cursor.execute("""
CREATE TABLE silver_detections_v2 (
    frame_index bigint,
    capture_ts timestamp(6),
    camera_id varchar,
    store_id varchar,
    det_id varchar,
    class varchar,
    class_id integer,
    conf double,
    bbox_x1 double,
    bbox_y1 double,
    bbox_x2 double,
    bbox_y2 double,
    centroid_x double,
    centroid_y double,
    track_id bigint,
    ingest_ts timestamp(6)
)
WITH (
    format = 'PARQUET'
)
""")
print("created silver_detections_v2")


TrinoExternalError: TrinoExternalError(type=EXTERNAL, name=ICEBERG_CATALOG_ERROR, message="Failed to load table: rva.silver_detections_v2", query_id=20251124_122558_00096_7v8pb)

#### Insert dữ liệu từ Bronze → Silver

In [ ]:
cursor.execute("""
INSERT INTO silver_detections_v2
WITH base AS (
    SELECT
        CAST(json_extract_scalar(payload, '$.frame_index') AS bigint) AS frame_index,
        CAST(from_iso8601_timestamp(json_extract_scalar(payload, '$.capture_ts')) AS timestamp(6)) AS capture_ts,
        camera_id,
        store_id,
        json_extract_scalar(det, '$.det_id') AS det_id,
        json_extract_scalar(det, '$.class') AS class,
        CAST(json_extract_scalar(det, '$.class_id') AS integer) AS class_id,
        CAST(json_extract_scalar(det, '$.conf') AS double) AS conf,
        CAST(json_extract_scalar(det, '$.bbox.x1') AS double) AS bbox_x1,
        CAST(json_extract_scalar(det, '$.bbox.y1') AS double) AS bbox_y1,
        CAST(json_extract_scalar(det, '$.bbox.x2') AS double) AS bbox_x2,
        CAST(json_extract_scalar(det, '$.bbox.y2') AS double) AS bbox_y2,
        CAST(json_extract_scalar(det, '$.centroid.x') AS double) AS centroid_x,
        CAST(json_extract_scalar(det, '$.centroid.y') AS double) AS centroid_y,
        CAST(json_extract_scalar(det, '$.track_id') AS bigint) AS track_id,
        CAST(ingest_ts AS timestamp(6)) AS ingest_ts
    FROM bronze_raw
    CROSS JOIN UNNEST(
        CAST(json_extract(payload, '$.detections') AS array(json))
    ) AS t(det)
),
clean AS (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY store_id, camera_id, frame_index,
                         COALESCE(det_id, CAST(track_id AS varchar))
            ORDER BY conf DESC, ingest_ts DESC
        ) AS rn
    FROM base
    WHERE
        frame_index IS NOT NULL
        AND capture_ts IS NOT NULL
        AND camera_id IS NOT NULL
        AND store_id IS NOT NULL
        AND det_id IS NOT NULL
        AND track_id IS NOT NULL
        AND conf IS NOT NULL
        AND conf >= 0.4
)
SELECT
    frame_index,
    capture_ts,
    camera_id,
    store_id,
    det_id,
    class,
    class_id,
    conf,
    bbox_x1,
    bbox_y1,
    bbox_x2,
    bbox_y2,
    centroid_x,
    centroid_y,
    track_id,
    ingest_ts
FROM clean
WHERE rn = 1
""")
print("inserted into silver_detections_v2")


inserted into silver_detections_v2


#### Kiểm tra lại

In [ ]:
cursor.execute("SELECT COUNT(*) FROM silver_detections_v2")
print(cursor.fetchall())

cursor.execute("""
SELECT * FROM silver_detections_v2
ORDER BY capture_ts
LIMIT 10
""")
print(cursor.fetchall())


[[1499]]
[[1, datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), 'cam_01', 'store_01', '1-2', 'person', 0, 0.8445228338241577, 0.01546478271484375, 223.85882568359375, 66.65916442871094, 503.38714599609375, 33.0, 363.0, 3, datetime.datetime(2025, 11, 22, 13, 10, 5, 546000)], [1, datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), 'cam_01', 'store_01', '1-0', 'person', 0, 0.879061222076416, 547.5609130859375, 208.7528076171875, 647.697265625, 566.1458740234375, 597.0, 387.0, 1, datetime.datetime(2025, 11, 22, 13, 10, 5, 546000)], [1, datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), 'cam_01', 'store_01', '1-1', 'person', 0, 0.8788681030273438, 720.4228515625, 106.02847290039062, 808.581298828125, 287.6534729003906, 764.0, 196.0, 2, datetime.datetime(2025, 11, 22, 13, 10, 5, 546000)], [1, datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), 'cam_01', 'store_01', '1-3', 'person', 0, 0.8155996799468994, 926.5863037109375, 0.5677490234375, 1012.654052734375, 214.39083862304688, 969.0,

#### Khám phá dữ liệu (silver)

##### 1.1 – Tổng số detection, frame, track

In [ ]:
cursor.execute("""
SELECT
    COUNT(*) AS total_detections,
    COUNT(DISTINCT frame_index) AS total_frames,
    COUNT(DISTINCT track_id) AS total_tracks
FROM silver_detections_v2
""")
print(cursor.fetchall())


[[1499, 317, 8]]


##### 1.2 – Kiểm tra missing frame (frame bị nhảy)

In [ ]:
cursor.execute("""
WITH f AS (
    SELECT DISTINCT frame_index
    FROM silver_detections_v2
)
SELECT
    MIN(frame_index) AS min_frame,
    MAX(frame_index) AS max_frame,
    COUNT(*) AS frame_count,
    (MAX(frame_index) - MIN(frame_index) + 1) AS expected_frames,
    (MAX(frame_index) - MIN(frame_index) + 1) - COUNT(*) AS missing_frames
FROM f
""")
print(cursor.fetchall())


[[1, 317, 317, 317, 0]]


#### 1.3 – Kiểm tra số người mỗi frame (để xem có outlier)

In [ ]:
cursor.execute("""
SELECT
    frame_index,
    COUNT(*) AS num_person
FROM silver_detections_v2
GROUP BY frame_index
ORDER BY frame_index
LIMIT 20
""")
print(cursor.fetchall())


[[1, 4], [2, 4], [3, 4], [4, 4], [5, 4], [6, 4], [7, 4], [8, 4], [9, 4], [10, 4], [11, 4], [12, 4], [13, 4], [14, 4], [15, 4], [16, 4], [17, 4], [18, 4], [19, 4], [20, 4]]


#### 1.4 – Kiểm tra phân bố confidence (độ tự tin của YOLO)

In [ ]:
cursor.execute("""
SELECT
    approx_percentile(conf, 0.1) AS p10,
    approx_percentile(conf, 0.5) AS p50,
    approx_percentile(conf, 0.9) AS p90
FROM silver_detections_v2
""")
print(cursor.fetchall())


[[0.6662853403271032, 0.8361939502049104, 0.8869955837726593]]


##### 1.5 – Kiểm tra timestamp đều hay không

In [ ]:
cursor.execute("""
WITH t AS (
    SELECT
        capture_ts,
        LAG(capture_ts) OVER (ORDER BY capture_ts) AS prev_ts,
        ROW_NUMBER() OVER (ORDER BY capture_ts) AS rn
    FROM silver_detections_v2
)
SELECT
    capture_ts,
    prev_ts,
    (capture_ts - prev_ts) AS gap
FROM t
WHERE rn <= 20
""")
print(cursor.fetchall())


[[datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.timedelta(0)], [datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), None, None], [datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.timedelta(0)], [datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.timedelta(0)], [datetime.datetime(2025, 11, 21, 19, 48, 51, 165000), datetime.datetime(2025, 11, 21, 19, 48, 51, 165000), datetime.timedelta(0)], [datetime.datetime(2025, 11, 21, 19, 48, 51, 165000), datetime.datetime(2025, 11, 21, 19, 48, 51, 165000), datetime.timedelta(0)], [datetime.datetime(2025, 11, 21, 19, 48, 51, 165000), datetime.datetime(2025, 11, 21, 19, 48, 51, 165000), datetime.timedelta(0)], [datetime.datetime(2025, 11, 21, 19, 48, 51, 165000), datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.timedelta(microseconds=258000)

#### Bước 2 – Phân tích track_id (hành vi từng người)

##### 2.1. Thống kê mỗi track_id

In [ ]:
cursor.execute("""
SELECT
    track_id,
    COUNT(*) AS frames_visible,
    MIN(capture_ts) AS start_time,
    MAX(capture_ts) AS end_time,
    MAX(capture_ts) - MIN(capture_ts) AS duration
FROM silver_detections_v2
GROUP BY track_id
ORDER BY track_id
""")
print(cursor.fetchall())


[[1, 317, datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.datetime(2025, 11, 21, 19, 49, 25, 843000), datetime.timedelta(seconds=34, microseconds=936000)], [2, 26, datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.datetime(2025, 11, 21, 19, 48, 54, 649000), datetime.timedelta(seconds=3, microseconds=742000)], [3, 311, datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.datetime(2025, 11, 21, 19, 49, 25, 301000), datetime.timedelta(seconds=34, microseconds=394000)], [4, 317, datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.datetime(2025, 11, 21, 19, 49, 25, 843000), datetime.timedelta(seconds=34, microseconds=936000)], [5, 15, datetime.datetime(2025, 11, 21, 19, 48, 54, 695000), datetime.datetime(2025, 11, 21, 19, 48, 55, 527000), datetime.timedelta(microseconds=832000)], [6, 280, datetime.datetime(2025, 11, 21, 19, 48, 55, 346000), datetime.datetime(2025, 11, 21, 19, 49, 25, 843000), datetime.timedelta(seconds=30, microseconds=497000)], 

##### 2.2. Vị trí trung bình mỗi người (để sau này dựa vào làm heatmap / zone)

In [ ]:
cursor.execute("""
SELECT
    track_id,
    AVG(centroid_x) AS avg_x,
    AVG(centroid_y) AS avg_y
FROM silver_detections_v2
GROUP BY track_id
ORDER BY track_id
""")
print(cursor.fetchall())


[[1, 382.3659305993691, 392.8706624605678], [2, 730.0384615384615, 207.34615384615384], [3, 35.430868167202576, 368.16720257234726], [4, 967.9873817034701, 106.93690851735016], [5, 108.66666666666667, 178.6], [6, 319.39285714285717, 196.60357142857143], [7, 126.95238095238095, 224.03896103896105], [8, 170.0, 152.5]]


##### 2.3. Di chuyển (độ lệch x, y) mỗi track

In [ ]:
cursor.execute("""
SELECT
    track_id,
    MIN(centroid_x) AS min_x,
    MAX(centroid_x) AS max_x,
    MAX(centroid_x) - MIN(centroid_x) AS delta_x,
    MIN(centroid_y) AS min_y,
    MAX(centroid_y) AS max_y,
    MAX(centroid_y) - MIN(centroid_y) AS delta_y
FROM silver_detections_v2
GROUP BY track_id
ORDER BY track_id
""")
print(cursor.fetchall())


[[1, 108.0, 597.0, 489.0, 335.0, 465.0, 130.0], [2, 682.0, 764.0, 82.0, 196.0, 224.0, 28.0], [3, 27.0, 47.0, 20.0, 351.0, 476.0, 125.0], [4, 956.0, 990.0, 34.0, 105.0, 108.0, 3.0], [5, 105.0, 113.0, 8.0, 172.0, 187.0, 15.0], [6, 132.0, 636.0, 504.0, 137.0, 234.0, 97.0], [7, 91.0, 168.0, 77.0, 173.0, 278.0, 105.0], [8, 170.0, 170.0, 0.0, 152.0, 153.0, 1.0]]


#### BƯỚC 3 – TẠO HEATMAP

##### 3.1 – SQL tạo heatmap grid 10×10

In [ ]:
cursor.execute("""
WITH grid AS (
    SELECT
        floor(centroid_x / (1280 / 10)) AS gx,
        floor(centroid_y / (720 / 10)) AS gy
    FROM silver_detections_v2
)
SELECT
    gx, gy,
    COUNT(*) AS hits
FROM grid
GROUP BY gx, gy
ORDER BY gy, gx
""")
rows = cursor.fetchall()
for r in rows:
    print(r)


[4.0, 1.0, 6]
[7.0, 1.0, 317]
[0.0, 2.0, 66]
[1.0, 2.0, 111]
[2.0, 2.0, 101]
[3.0, 2.0, 43]
[4.0, 2.0, 13]
[5.0, 2.0, 16]
[0.0, 3.0, 97]
[1.0, 3.0, 74]
[2.0, 3.0, 17]
[5.0, 3.0, 10]
[0.0, 4.0, 144]
[1.0, 4.0, 58]
[0.0, 5.0, 193]
[1.0, 5.0, 14]
[3.0, 5.0, 9]
[4.0, 5.0, 147]
[0.0, 6.0, 15]
[1.0, 6.0, 18]
[2.0, 6.0, 18]
[3.0, 6.0, 12]


#### BƯỚC 4 – PHÂN TÍCH DWELL TIME (người đứng bao lâu)

In [ ]:
cursor.execute("""
WITH grid AS (
    SELECT
        track_id,
        floor(centroid_x / (1280 / 10)) AS gx,
        floor(centroid_y / (720 / 10)) AS gy,
        capture_ts
    FROM silver_detections_v2
),
dwell AS (
    SELECT
        track_id,
        gx,
        gy,
        MIN(capture_ts) AS start_time,
        MAX(capture_ts) AS end_time,
        MAX(capture_ts) - MIN(capture_ts) AS duration
    FROM grid
    GROUP BY track_id, gx, gy
)
SELECT *
FROM dwell
ORDER BY duration DESC
LIMIT 20
""")
print(cursor.fetchall())


[[4, 7.0, 1.0, datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.datetime(2025, 11, 21, 19, 49, 25, 843000), datetime.timedelta(seconds=34, microseconds=936000)], [3, 0.0, 5.0, datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.datetime(2025, 11, 21, 19, 49, 24, 99000), datetime.timedelta(seconds=33, microseconds=192000)], [3, 0.0, 4.0, datetime.datetime(2025, 11, 21, 19, 48, 52, 814000), datetime.datetime(2025, 11, 21, 19, 49, 21, 935000), datetime.timedelta(seconds=29, microseconds=121000)], [7, 0.0, 2.0, datetime.datetime(2025, 11, 21, 19, 48, 59, 330000), datetime.datetime(2025, 11, 21, 19, 49, 21, 241000), datetime.timedelta(seconds=21, microseconds=911000)], [1, 4.0, 5.0, datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.datetime(2025, 11, 21, 19, 49, 9, 800000), datetime.timedelta(seconds=18, microseconds=893000)], [6, 2.0, 2.0, datetime.datetime(2025, 11, 21, 19, 49, 3, 410000), datetime.datetime(2025, 11, 21, 19, 49, 17, 575000), datetime.tim

#### BƯỚC 5 – PATTERN THEO THỜI GIAN (Temporal Analytics)

Trong bước này ta phân tích:

số người mỗi giây

số người mỗi frame

peak time

flow direction (trend di chuyển theo thời gian)

timeline của từng track

##### 5.1 – Số người theo giây

In [ ]:
cursor.execute("""
SELECT
    date_trunc('second', capture_ts) AS sec,
    COUNT(*) AS detections,
    COUNT(DISTINCT track_id) AS unique_people
FROM silver_detections_v2
GROUP BY 1
ORDER BY 1
""")
print(cursor.fetchall())


[[datetime.datetime(2025, 11, 21, 19, 48, 50), 4, 4], [datetime.datetime(2025, 11, 21, 19, 48, 51), 28, 4], [datetime.datetime(2025, 11, 21, 19, 48, 52), 24, 4], [datetime.datetime(2025, 11, 21, 19, 48, 53), 24, 4], [datetime.datetime(2025, 11, 21, 19, 48, 54), 48, 5], [datetime.datetime(2025, 11, 21, 19, 48, 55), 64, 5], [datetime.datetime(2025, 11, 21, 19, 48, 56), 48, 4], [datetime.datetime(2025, 11, 21, 19, 48, 57), 20, 4], [datetime.datetime(2025, 11, 21, 19, 48, 58), 20, 4], [datetime.datetime(2025, 11, 21, 19, 48, 59), 28, 5], [datetime.datetime(2025, 11, 21, 19, 49), 35, 5], [datetime.datetime(2025, 11, 21, 19, 49, 1), 40, 5], [datetime.datetime(2025, 11, 21, 19, 49, 2), 35, 5], [datetime.datetime(2025, 11, 21, 19, 49, 3), 30, 5], [datetime.datetime(2025, 11, 21, 19, 49, 4), 40, 5], [datetime.datetime(2025, 11, 21, 19, 49, 5), 40, 5], [datetime.datetime(2025, 11, 21, 19, 49, 6), 40, 5], [datetime.datetime(2025, 11, 21, 19, 49, 7), 35, 5], [datetime.datetime(2025, 11, 21, 19, 49

##### 5.2 – Hướng di chuyển theo thời gian (x movement)

In [ ]:
cursor.execute("""
SELECT
    track_id,
    MIN(centroid_x) AS start_x,
    MAX(centroid_x) AS end_x,
    MAX(centroid_x) - MIN(centroid_x) AS movement_x
FROM silver_detections_v2
GROUP BY track_id
ORDER BY track_id
""")
print(cursor.fetchall())


[[1, 108.0, 597.0, 489.0], [2, 682.0, 764.0, 82.0], [3, 27.0, 47.0, 20.0], [4, 956.0, 990.0, 34.0], [5, 105.0, 113.0, 8.0], [6, 132.0, 636.0, 504.0], [7, 91.0, 168.0, 77.0], [8, 170.0, 170.0, 0.0]]


##### 5.3 – Timeline của từng track

In [ ]:
cursor.execute("""
SELECT
    track_id,
    MIN(capture_ts),
    MAX(capture_ts),
    MAX(capture_ts) - MIN(capture_ts) AS duration
FROM silver_detections_v2
GROUP BY track_id
ORDER BY track_id
""")
print(cursor.fetchall())


[[1, datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.datetime(2025, 11, 21, 19, 49, 25, 843000), datetime.timedelta(seconds=34, microseconds=936000)], [2, datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.datetime(2025, 11, 21, 19, 48, 54, 649000), datetime.timedelta(seconds=3, microseconds=742000)], [3, datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.datetime(2025, 11, 21, 19, 49, 25, 301000), datetime.timedelta(seconds=34, microseconds=394000)], [4, datetime.datetime(2025, 11, 21, 19, 48, 50, 907000), datetime.datetime(2025, 11, 21, 19, 49, 25, 843000), datetime.timedelta(seconds=34, microseconds=936000)], [5, datetime.datetime(2025, 11, 21, 19, 48, 54, 695000), datetime.datetime(2025, 11, 21, 19, 48, 55, 527000), datetime.timedelta(microseconds=832000)], [6, datetime.datetime(2025, 11, 21, 19, 48, 55, 346000), datetime.datetime(2025, 11, 21, 19, 49, 25, 843000), datetime.timedelta(seconds=30, microseconds=497000)], [7, datetime.datetime(2025, 

### Lớp Gold

#### Bảng Gold 1: gold_people_per_minute
Ý nghĩa:
Mỗi dòng = 1 phút / 1 camera / 1 store:

có bao nhiêu detection (số bounding box)

có bao nhiêu người unique (track_id)

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS gold_people_per_minute (
    store_id varchar,
    camera_id varchar,
    ts_minute timestamp(6),
    detections bigint,
    unique_people bigint
)
WITH (format = 'PARQUET')
""")
print("Created gold_people_per_minute")


Created gold_people_per_minute


In [ ]:
cursor.execute("""
INSERT INTO gold_people_per_minute
SELECT
    store_id,
    camera_id,
    date_trunc('minute', capture_ts) AS ts_minute,
    COUNT(*) AS detections,
    COUNT(DISTINCT track_id) AS unique_people
FROM silver_detections_v2
GROUP BY store_id, camera_id, date_trunc('minute', capture_ts)
""")
print("Inserted gold_people_per_minute")


Inserted gold_people_per_minute


#### Bảng Gold 2: gold_zone_heatmap
Ý nghĩa:
Heatmap tổng hợp: mỗi dòng = 1 zone (ô lưới) trong khung hình + số lần xuất hiện:

dùng cho heatmap overlay

cluster xem khu vực nào đông khách

Ta tiếp tục dùng lưới 10×10 (như lúc nãy):

zone_x = 0..9 (trái → phải)

zone_y = 0..9 (trên → dưới)

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS gold_zone_heatmap (
    store_id varchar,
    camera_id varchar,
    zone_x integer,
    zone_y integer,
    hits bigint,
    unique_tracks bigint
)
WITH (format = 'PARQUET')
""")
print("Created gold_zone_heatmap")


Created gold_zone_heatmap


In [ ]:
cursor.execute("""
INSERT INTO gold_zone_heatmap
SELECT
    store_id,
    camera_id,
    CAST(floor(centroid_x / (1280 / 10)) AS integer) AS zone_x,
    CAST(floor(centroid_y / (720 / 10)) AS integer) AS zone_y,
    COUNT(*) AS hits,
    COUNT(DISTINCT track_id) AS unique_tracks
FROM silver_detections_v2
GROUP BY
    store_id,
    camera_id,
    CAST(floor(centroid_x / (1280 / 10)) AS integer),
    CAST(floor(centroid_y / (720 / 10)) AS integer)
""")
print("Inserted gold_zone_heatmap")


Inserted gold_zone_heatmap


#### Bảng Gold 3: gold_zone_dwell

Ý nghĩa:
Đây là bảng rất “xịn”:

mỗi dòng = 1 store + camera + zone

chứa thông tin thời gian khách đứng lại (dwell)

Từ Silver mình đã tính dwell theo track_id + zone_x + zone_y.
Giờ Gold sẽ tổng hợp lại:

tổng thời gian đứng (sum_dwell_seconds)

dwell trung bình (avg_dwell_seconds)

số lượt (số track) vào zone (visits)

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS gold_zone_dwell (
    store_id varchar,
    camera_id varchar,
    zone_x integer,
    zone_y integer,
    visits bigint,
    total_dwell_seconds double,
    avg_dwell_seconds double
)
WITH (format = 'PARQUET')
""")
print("Created gold_zone_dwell")


Created gold_zone_dwell


In [ ]:
cursor.execute("""
INSERT INTO gold_zone_dwell
WITH per_track_zone AS (
    SELECT
        store_id,
        camera_id,
        track_id,
        CAST(floor(centroid_x / (1280 / 10)) AS integer) AS zone_x,
        CAST(floor(centroid_y / (720 / 10)) AS integer) AS zone_y,
        MIN(capture_ts) AS start_time,
        MAX(capture_ts) AS end_time,
        date_diff('millisecond', MIN(capture_ts), MAX(capture_ts)) / 1000.0 AS dwell_seconds
    FROM silver_detections_v2
    GROUP BY
        store_id,
        camera_id,
        track_id,
        CAST(floor(centroid_x / (1280 / 10)) AS integer),
        CAST(floor(centroid_y / (720 / 10)) AS integer)
)
SELECT
    store_id,
    camera_id,
    zone_x,
    zone_y,
    COUNT(*) AS visits,
    SUM(dwell_seconds) AS total_dwell_seconds,
    AVG(dwell_seconds) AS avg_dwell_seconds
FROM per_track_zone
GROUP BY store_id, camera_id, zone_x, zone_y
""")
print("Inserted gold_zone_dwell")


Inserted gold_zone_dwell


#### Thiết kế bảng gold_track_summary
Schema đề xuất:

store_id – cửa hàng

camera_id – camera

track_id – id của người/track

frames – số frame xuất hiện

start_time, end_time – thời gian bắt đầu/kết thúc

duration_seconds – thời gian tồn tại của track

min_x, max_x, delta_x – biên độ di chuyển theo chiều ngang

min_y, max_y, delta_y – biên độ di chuyển theo chiều dọc

avg_x, avg_y – vị trí trung bình (điểm trung tâm “quen thuộc”)

avg_conf – confidence trung bình của YOLO cho track đó

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS gold_track_summary (
    store_id varchar,
    camera_id varchar,
    track_id bigint,
    frames bigint,
    start_time timestamp(6),
    end_time timestamp(6),
    duration_seconds double,
    min_x double,
    max_x double,
    delta_x double,
    min_y double,
    max_y double,
    delta_y double,
    avg_x double,
    avg_y double,
    avg_conf double
)
WITH (format = 'PARQUET')
""")
print("Created gold_track_summary")


Created gold_track_summary


In [ ]:
cursor.execute("""
INSERT INTO gold_track_summary
SELECT
    store_id,
    camera_id,
    CAST(track_id AS bigint) AS track_id,
    COUNT(*) AS frames,
    MIN(capture_ts) AS start_time,
    MAX(capture_ts) AS end_time,
    date_diff('millisecond', MIN(capture_ts), MAX(capture_ts)) / 1000.0 AS duration_seconds,
    MIN(centroid_x) AS min_x,
    MAX(centroid_x) AS max_x,
    MAX(centroid_x) - MIN(centroid_x) AS delta_x,
    MIN(centroid_y) AS min_y,
    MAX(centroid_y) AS max_y,
    MAX(centroid_y) - MIN(centroid_y) AS delta_y,
    AVG(centroid_x) AS avg_x,
    AVG(centroid_y) AS avg_y,
    AVG(conf) AS avg_conf
FROM silver_detections_v2
GROUP BY
    store_id,
    camera_id,
    CAST(track_id AS bigint)
""")
print("Inserted gold_track_summary")


Inserted gold_track_summary


##### query kiểm tra nhanh 

In [ ]:
# Top track đứng lâu nhất
cursor.execute("""
SELECT track_id, duration_seconds
FROM gold_track_summary
ORDER BY duration_seconds DESC
LIMIT 10
""")
print(cursor.fetchall())

# Track di chuyển xa nhất theo trục X
cursor.execute("""
SELECT track_id, delta_x, delta_y
FROM gold_track_summary
ORDER BY delta_x DESC
LIMIT 10
""")
print(cursor.fetchall())


[[4, 34.9], [1, 34.9], [3, 34.4], [6, 30.5], [7, 26.5], [2, 3.7], [5, 0.8], [8, 0.1]]
[[6, 504.0, 97.0], [1, 489.0, 130.0], [2, 82.0, 28.0], [7, 77.0, 105.0], [4, 34.0, 3.0], [3, 20.0, 125.0], [5, 8.0, 15.0], [8, 0.0, 1.0]]
